# AI Planning

AI Planning explores the process of using autonomous techniques to solve planning and scheduling problems. A planning problem is one in which we have some initial starting state, which we wish to transform into a desired goal state through the application of a set of actions.

In [52]:
import random
import math
from PIL import Image
import numpy as np

## Forward State Space Planning

### Example: Robotic Arm Planning

You are simulating a robot arm inside a grid that contains ingredients. The goal is for the robot arm to pick up all the relevant ingredients (I) in the grid to create a drink.

**State**
- Grid layout with ingredients, either an irrelevant (0) or relevant ingredient (I)
- Robotic Arm's coordinates (x, y)

**Goal**: Pick up all ingredients one by one.

**Actions:** Move UP, DOWN, LEFT, RIGHT for navigation, and PICK UP ingredient

In [53]:
def forward_planning(grid, robot_pos):
    plan = []

    while any('I' in row for row in grid):
        closest_ingredient = find_closest_ingredient(grid, robot_pos)
        path = find_path_to_ingredient(grid, robot_pos, closest_ingredient)

        for action in path:
            plan.append(action)

        plan.append('PICK_UP')
        grid[closest_ingredient[0]][closest_ingredient[1]] = '0'
        robot_pos = closest_ingredient

    return plan

def find_closest_from_positions(last_pos, ingredients_positions):
    closest_distance = float('inf')
    closest_ingredient = None

    for pos in ingredients_positions:
        dist = abs(last_pos[0] - pos[0]) + abs(last_pos[1] - pos[1])
        if dist < closest_distance:
            closest_distance = dist
            closest_ingredient = pos

    return closest_ingredient

    return closest_ingredient

def find_path_to_ingredient(grid, start, end):
    moves = [(0, 1, 'RIGHT'), (1, 0, 'DOWN'), (0, -1, 'LEFT'), (-1, 0, 'UP')]
    queue = [(start, [])]
    visited = set()

    while queue:
        (current_pos, path) = queue.pop(0)  # Dequeue the front

        if current_pos == end:
            return path

        visited.add(current_pos)

        for dx, dy, action in moves:
            new_pos = (current_pos[0] + dx, current_pos[1] + dy)
            
            if 0 <= new_pos[0] < len(grid) and 0 <= new_pos[1] < len(grid[0]) and new_pos not in visited:
                new_path = path + [action]
                queue.append((new_pos, new_path))

    return []

In [54]:
grid = [
    ['0', '0', 'I', '0'],
    ['0', '0', '0', '0'],
    ['0', 'I', '0', 'I'],
    ['I', '0', '0', '0']
]

robot_pos = (0, 0)

# Retrieve Plan
plan = forward_planning(grid, robot_pos)
print(plan)

['RIGHT', 'RIGHT', 'PICK_UP', 'DOWN', 'DOWN', 'LEFT', 'PICK_UP', 'RIGHT', 'RIGHT', 'PICK_UP', 'DOWN', 'LEFT', 'LEFT', 'LEFT', 'PICK_UP']


## Backward State Space Planning

Backward planning, often known as "goal decomposition," starts with the goal and works backward to the initial state. Let's illustrate using the same goal and actions.

**Goal**: The robot returns all ingredients back to its initial position.

**Preconditions:** for Achieving the Goal:
- The robot's initial position is at the drop-off point with the ingredient
- Before being at the drop-off, the robot had the ingredient's initial location
- Before picking up an ingredient, the robot must have moved to its location.


In [65]:
def backward_planning(grid, robot_pos, forward_plan):
    plan = []

    # Reverse the order of the forward plan and iterate over it
    for action in reversed(forward_plan):
        if action == 'PICK_UP':
            plan.append('DROP')
        else:
            # For movement actions, we need to reverse the movement to undo it
            reverse_actions = {'UP': 'DOWN', 'DOWN': 'UP', 'LEFT': 'RIGHT', 'RIGHT': 'LEFT'}
            reverse_action = reverse_actions[action]
            plan.append(reverse_action)
            move_dict = {'UP': (-1, 0), 'DOWN': (1, 0), 'LEFT': (0, -1), 'RIGHT': (0, 1)}
            move = move_dict[reverse_action]
            robot_pos = (robot_pos[0] + move[0], robot_pos[1] + move[1])

    return plan

grid = [
    ['0', '0', 'I', '0'],
    ['0', '0', '0', '0'],
    ['0', 'I', '0', 'I'],
    ['I', '0', '0', '0']
]
robot_pos = (0, 0)

forward_plan = forward_planning([row.copy() for row in grid], robot_pos)
backward_plan = backward_planning([row.copy() for row in grid], robot_pos, forward_plan)

print(backward_plan)

['DROP', 'RIGHT', 'RIGHT', 'RIGHT', 'UP', 'DROP', 'LEFT', 'LEFT', 'DROP', 'RIGHT', 'UP', 'UP', 'DROP', 'LEFT', 'LEFT']
